In [ ]:
import pandas as pd
import numpy as np

from pathlib import Path
import sys
parent_dir = Path().resolve().parent
sys.path.insert(0, str(parent_dir))
from libs.functions import *

In [ ]:
df = pd.read_csv('data/predictions.csv')
df.rename(columns={'Unnamed: 0': 'index_original'}, inplace=True)
df

,index,period,timestamp,ticker,y_true,discount,Naïve,OLS,LASSO,MLP,MLP-Pyr,discount_month,y_true_full,Naïve_full,OLS_full,LASSO_full,MLP_full,MLP-Pyr_full
0,228,21,2021-01-31,AAB.CO,0.004785,0.0000,0.008812,0.014568,0.026861,0.017427,0.018745,0.000000,0.004785,0.008812,0.014568,0.026861,0.017427,0.018745
1,229,21,2021-02-28,AAB.CO,-0.028571,0.0000,0.008812,0.022468,0.028238,0.017074,0.018563,0.000000,-0.028571,0.008812,0.022468,0.028238,0.017074,0.018563
2,230,21,2021-03-31,AAB.CO,-0.009804,0.0000,0.008812,0.041455,0.025399,0.017697,0.021412,0.000000,-0.009804,0.008812,0.041455,0.025399,0.017697,0.021412
3,231,21,2021-04-30,AAB.CO,0.084158,0.0000,0.008812,0.032088,0.024087,0.014637,0.008495,0.000000,0.084158,0.008812,0.032088,0.024087,0.014637,0.008495
4,232,21,2021-05-31,AAB.CO,-0.068493,0.0000,0.008812,0.028618,0.021055,0.011121,0.005391,0.000000,-0.068493,0.008812,0.028618,0.021055,0.011121,0.005391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5954,45570,24,2024-08-31,ZELA.CO,-0.085660,0.0335,0.008954,0.012422,0.046373,0.015811,0.013228,0.002750,-0.082910,0.011704,0.015172,0.049123,0.018561,0.015977
5955,45571,24,2024-09-30,ZELA.CO,-0.030425,0.0310,0.008954,-0.051152,0.038206,0.016322,0.009828,0.002547,-0.027878,0.011502,-0.048605,0.040753,0.018869,0.012375
5956,45572,24,2024-10-31,ZELA.CO,-0.079081,0.0285,0.008954,-0.050271,0.039111,0.018203,0.010132,0.002344,-0.076737,0.011299,-0.047927,0.041455,0.020548,0.012477
5957,45573,24,2024-11-30,ZELA.CO,-0.022208,0.0285,0.008954,-0.001312,0.031701,0.016338,0.003199,0.002344,-0.019863,0.011299,0.001032,0.034046,0.018682,0.005544


In [ ]:
df['decile_ols'] = df.groupby(['timestamp'])['OLS'].transform(deciles)
df['decile_mlp'] = df.groupby(['timestamp'])['MLP'].transform(deciles)
df['decile_lasso'] = df.groupby(['timestamp'])['LASSO'].transform(deciles)
df['decile_mlp-pyr'] = df.groupby(['timestamp'])['MLP-Pyr'].transform(deciles)

display(df.sort_values(by=['timestamp', 'decile_mlp']))

,index,period,timestamp,ticker,y_true,discount,Naïve,OLS,LASSO,MLP,...,y_true_full,Naïve_full,OLS_full,LASSO_full,MLP_full,MLP-Pyr_full,decile_mlp,decile_ols,decile_lasso,decile_mlp-pyr
337,3586,21,2021-01-31,ATLA.CO,0.025000,0.000,0.008812,-0.068114,-0.006517,-0.017047,...,0.025000,0.008812,-0.068114,-0.006517,-0.017047,-0.048342,0,0,0,0
385,4033,21,2021-01-31,BAVA.CO,0.005460,0.000,0.008812,0.020622,0.005974,-0.026621,...,0.005460,0.008812,0.020622,0.005974,-0.026621,0.002771,0,3,0,0
483,4797,21,2021-01-31,BIOPOR.CO,0.006515,0.000,0.008812,0.147594,0.006574,-0.005518,...,0.006515,0.008812,0.147594,0.006574,-0.005518,-0.006712,0,9,0,0
961,8208,21,2021-01-31,CPHCAPST.CO,0.055556,0.000,0.008812,0.032147,0.011804,0.000843,...,0.055556,0.008812,0.032147,0.011804,0.000843,0.007493,0,4,0,1
1510,12579,21,2021-01-31,ERRIA.CO,0.178010,0.000,0.008812,0.032338,-0.000547,-0.009530,...,0.178010,0.008812,0.032338,-0.000547,-0.009530,-0.011056,0,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2829,22308,24,2024-12-31,KLEEb.CO,0.040217,0.026,0.008954,-0.023926,0.031326,0.017362,...,0.042359,0.011095,-0.021784,0.033467,0.019503,0.009795,9,1,9,8
3281,26287,24,2024-12-31,MTHH.CO,-0.023106,0.026,0.008954,0.062999,0.053678,0.022435,...,-0.020965,0.011095,0.065140,0.055819,0.024577,0.023204,9,9,9,9
4204,32238,24,2024-12-31,PENNEO.CO^C25,0.025957,0.026,0.008954,0.010678,0.020090,0.019670,...,0.028099,0.011095,0.012819,0.022231,0.021812,0.019755,9,3,7,9
5066,37957,24,2024-12-31,SKAKO.CO,0.027212,0.026,0.008954,0.014789,0.033003,0.021562,...,0.029353,0.011095,0.016930,0.035145,0.023703,0.020282,9,4,9,9
